In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('../scraping/cleansed_data/players_20230425190937.csv', sep=';')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18335 entries, 0 to 18334
Data columns (total 48 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   LeagueCountry             18335 non-null  object 
 1   League                    18238 non-null  object 
 2   NationalLeagueLevel       18335 non-null  int64  
 3   Club                      18331 non-null  object 
 4   No                        17905 non-null  float64
 5   Name                      18331 non-null  object 
 6   Value                     17899 non-null  float64
 7   ValueLastUpdate           17899 non-null  object 
 8   HighestValue              18004 non-null  float64
 9   HighestValueDate          18004 non-null  object 
 10  ValueCurrency             17899 non-null  object 
 11  Age                       18328 non-null  float64
 12  Height                    17052 non-null  float64
 13  Nationality               18331 non-null  object 
 14  Foot  

In [35]:
null_values = df['Minutes'].isnull().groupby([df['LeagueCountry'], df['League']]).sum().to_frame()
size_values = df.groupby(['LeagueCountry', 'League']).size()

null_values['Players'] = size_values
null_values = null_values.rename(columns={'Minutes': 'NullValues'})
null_values['NullValuesRelative'] = null_values['NullValues'] / null_values['Players'] * 100

null_values = null_values[null_values['Players'] > 20]  # exclude minor leagues
null_values = null_values.sort_values('NullValuesRelative', ascending=False)

print(null_values)

                                     NullValues  Players  NullValuesRelative
LeagueCountry League                                                        
Türkei        Süper Lig                     520      520          100.000000
Niederlande   Eredivisie                    483      483          100.000000
Serbien       Omladinska liga                23       26           88.461538
Griechenland  Super League 2                616      828           74.396135
Norwegen      Eliteserien                   279      414           67.391304
Schottland    Championship                  155      239           64.853556
Schweiz       Promotion League              269      419           64.200477
Schweden      Allsvenskan                   266      438           60.730594
Spanien       LaLiga2                       300      531           56.497175
England       League One                    333      624           53.365385
Tschechien    Fortuna Liga                  162      407           39.803440